In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import Word2Vec
import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train=pd.read_csv("Training_set.csv")

In [3]:
train=train.sample(100000)

In [4]:
test=pd.read_csv("Test_set.csv")

In [5]:
def get_job_characteristics(df): 
    job_type=[]
    job_title=[]
    employer_title=[]
    employer_sector_title=[]
    job_sector_title=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        types=data["job_type"]
        title=data["job_title"]
        em_title=data["employer_title"]
        em_sec_title=data["employer_sector_title"]
        job_sec_title=data["job_sector_title"]
        job_type.append(types)
        job_title.append(title)
        employer_title.append(em_title)
        employer_sector_title.append(em_sec_title)
        job_sector_title.append(job_sec_title)
    d={}
    #d["job_title"]=job_title
    #d["employer_title"]=employer_title
    d["employer_sector_title"]=employer_sector_title
    d["job_sector_title"]=job_sector_title
    a=pd.DataFrame(d)
    data=pd.get_dummies(a)
    b=pd.get_dummies(job_type)
    result = pd.concat([data, b], axis=1, sort=False)
    return result

In [6]:
def get_data_for_user(user):
    jobtitles=list(user["job_title"].values)
    emtitles=list(user["employer_title"].values)
    jobsectitles=list(user["job_sector_title"].values)
    emsectitles=list(user["employer_sector_title"].values)
    a=""
    for k in range(len(jobtitles)):
        a+=jobtitles[k]+" "
        a+=emtitles[k]+" "
        a+=jobsectitles[k]+" "
        #a+=emsectitles[k]+" "
    return a

In [7]:
def get_user_characteristics(df):
    corpus=[]
    gender=[]
    ethnicity=[]
    school_type=[]
    right_to_work_uk=[]
    university_name=[]
    university_type=[]
    course_includes_industrial_placement=[]
    degree_subject_name=[]
    user_preference=[]
    job_types=list(set(df["job_type"].values))
    em_sectors=list(set(df["employer_sector_title"].values))
    job_sectors=list(set(df["job_sector_title"].values))
    count_job_types=[[] for i in range(len(job_types))]
    count_job_sectors=[[] for i in range(len(job_sectors))]
    count_em_sectors=[[] for i in range(len(em_sectors))]
    nb_em_sector=[]
    nb_job_sector=[]
    job_type=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        user=data["user_id"]
        user_data=df[df["user_id"]==user]
        user_job_type=dict(Counter(list(user_data["job_type"].values)))
        user_job_sectors=dict(Counter(list(user_data["job_sector_title"].values)))
        user_em_sectors=dict(Counter(list(user_data["employer_sector_title"].values)))
        user_em_sector=len(list(set(list(user_data["employer_sector_title"].values))))
        user_job_sector=len(list(set(list(user_data["job_sector_title"].values))))
        for i in range(len(count_job_types)):
            try:
                count_job_types[i].append(user_job_type[i])
            except:
                count_job_types[i].append(0)
        for i in range(len(count_job_sectors)):
            try:
                count_job_sectors[i].append(user_job_sectors[i])
            except:
                count_job_sectors[i].append(0)
        for i in range(len(count_em_sectors)):
            try:
                count_em_sectors[i].append(user_em_sectors[i])
            except:
                count_em_sectors[i].append(0)
        sex=data["gender"]
        jobtype=data["job_type"]
        eth=data["ethnicity"]
        school=data["school_type"]
        placement=data["course_includes_industrial_placement"]
        right=data["right_to_work_uk"]
        uni=data["university_name"]
        uni_type=data["university_type"]
        subname=data["degree_subject_name"]
        pref=data["user_preference"]
        title=data["job_title"]
        em_title=data["employer_title"]
        em_sec_title=data["employer_sector_title"]
        job_sec_title=data["job_sector_title"]
        if sex=="F":
            sex="Female"
        elif sex=="M":
            sex="Male"
        if placement==True:
            placement=1
        elif placement==False:
            placement=-1
        else:
            placement=0
        if right==True:
            right=1
        elif right==False:
            right=-1
        else:
            right=0
        if type(pref) is float:
            pref=""
        if type(subname) is float:
            subname=""
        nb_em_sector.append(user_em_sector)
        nb_job_sector.append(user_job_sector)
        gender.append(sex)
        ethnicity.append(eth)
        school_type.append(school)
        course_includes_industrial_placement.append(placement)
        right_to_work_uk.append(right)
        university_type.append(uni_type)
        #degree_subject_name.append(subname)
        #user_preference.append(pref)
        job_type.append(jobtype)
        corpus.append(pref+subname+" "+get_data_for_user(user_data))
    d={}
    for i in range(len(count_job_sectors)):
        d['count_job_sectors_'+str(i)]=count_job_sectors[i]
    for i in range(len(count_em_sectors)):
        d['count_em_sectors_'+str(i)]=count_em_sectors[i]
    for i in range(len(count_job_types)):
        d['count_job_types_'+str(i)]=count_job_types[i]
    d["gender"]=gender
    d["job_type"]=job_type
    d["nb_em_sector"]=nb_em_sector
    d["nb_job_sector"]=nb_job_sector
    d["right_to_work_uk"]=right_to_work_uk
    d["ethnicity"]=ethnicity
    #d["degree_subject_name"]=degree_subject_name
    d["school_type"]=school_type
    #d["user_preference"]=user_preference
    d["course_includes_industrial_placement"]=course_includes_industrial_placement
    a=pd.DataFrame(d)
    data=pd.get_dummies(a)
    b=pd.get_dummies(university_type)
    result = pd.concat([data, b], axis=1, sort=False)
    vectorizer = CountVectorizer(max_df=1.0, min_df=3)
    X = vectorizer.fit_transform(corpus)
    col=vectorizer.get_feature_names()
    donnees=X.toarray()
    print(X.shape)
    donnee=pd.DataFrame(donnees, columns=col)
    final=pd.concat([result, donnee], axis=1)
    return final

In [8]:
#user_characteristics=get_user_characteristics(train)

In [9]:
#user_characteristics.shape

In [10]:
#job_characteristics=get_job_characteristics(train)

In [11]:
#job_characteristics.shape

In [12]:
def get_user_info(df):
    user_info=[]
    d={}
    x=train[["user_id","job_title","employer_title","job_sector_title","job_description",
             "employer_sector_title","employer_description"]]
    n=df.shape[0]
    for i in range(n):
        if i%5000==0:
            print("progress: ",i/n)
        data=df.iloc[i]
        user_id=data["user_id"]
        try:
            a=d[str(user_id)]
        except:
            y=x[x["user_id"]==user_id]
            job_titles=y["job_title"].values
            job_sector_title=y["job_sector_title"].values
            description=y["job_description"].values
            em_titles=y["employer_title"].values
            em_sector_title=y["employer_sector_title"].values
            em_description=y["employer_description"].values
            university_name=data["university_name"]
            degree_subject_name=data["degree_subject_name"]
            user_preference=data["user_preference"]
            ethnicity=data["ethnicity"]
            school_type=data["school_type"]
            if type(user_preference) is float:
                user_preference=""
            if type(university_name) is float:
                university_name=""
            if type(degree_subject_name) is float:
                degree_subject_name=""
            if type(ethnicity) is float:
                ethnicity=""
            if type(school_type) is float:
                school_type=""
            user_preference=user_preference+" "
            user_preference=user_preference *3
            degree_subject_name=degree_subject_name+" "
            degree_subject_name=degree_subject_name *3
            a=""
            a+=university_name+" "
            a+=degree_subject_name+" "
            a+=user_preference+" "
            a+=ethnicity+" "
            a+=school_type+" "
            for k in range(len(job_titles)):
                a+=job_titles[k]+" "
                a+=job_sector_title[k]+" "
                a+=description[k]+" "
                a+=em_titles[k]+" "
                a+=em_sector_title[k]+" "
                a+=em_description[k]+" "
            a=a.replace("Graduate","").replace("Programme","").replace("  "," ").replace("&","")
            d[str(user_id)]=a
        user_info.append(a)
    return user_info

In [13]:
train_user_info=get_user_info(train)

progress:  0.0
progress:  0.05
progress:  0.1
progress:  0.15
progress:  0.2
progress:  0.25
progress:  0.3
progress:  0.35
progress:  0.4
progress:  0.45
progress:  0.5
progress:  0.55
progress:  0.6
progress:  0.65
progress:  0.7
progress:  0.75
progress:  0.8
progress:  0.85
progress:  0.9
progress:  0.95


In [14]:
len(train_user_info)

100000

In [15]:
def clean_user(x,df):
    n=len(x)
    res=[]
    d={}
    for i,content in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        data=df.iloc[i]
        user_id=data["user_id"]
        try:
            y=d[str(user_id)]
        except:
            content=content.replace(",","").replace("-"," ").replace("  "," ").replace("&eacute;","e")
            content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
            content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
            content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
            content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
            content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
            content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
            content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
            content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
            content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
            content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
            content=re.sub('<.*?>',"",content , flags=re.DOTALL)
            content=content.replace("&ldquo;","").replace("ldquo;","").replace("&rdquo;","").replace("rdquo;","")
            try:
                content=''.join([i for i in content if not i.isdigit()])
            except:
                pass
            content=content.split(" ")
            y=[]
            for i in content:
                if len(i)>1:
                    y.append(i.lower())
            d[str(user_id)]=y
        res.append(y)
    return res

In [16]:
clean_train_user_info=clean_user(train_user_info, train)

progress :  0.0
progress :  0.05
progress :  0.1
progress :  0.15
progress :  0.2
progress :  0.25
progress :  0.3
progress :  0.35
progress :  0.4
progress :  0.45
progress :  0.5
progress :  0.55
progress :  0.6
progress :  0.65
progress :  0.7
progress :  0.75
progress :  0.8
progress :  0.85
progress :  0.9
progress :  0.95


In [17]:
def get_job_info(df):
    job_info=[]
    d={}
    for i in range(df.shape[0]):
        data=df.iloc[i]
        job_id=data["job_id"]
        try:
            a=d[str(job_id)]
        except:
            title=data["job_title"]
            job_sector_title=data["job_sector_title"]
            description=data["job_description"]
            em_title=data["employer_title"]
            em_sector_title=data["employer_sector_title"]
            em_description=data["employer_description"]
            a=""
            a+=title+" "
            a+=job_sector_title+" "
            a+=description+" "
            a+=em_title+" "
            a+=em_sector_title+" "
            a+=em_description
            d[str(job_id)]=a
        job_info.append(a)
    return job_info

In [18]:
train_job_info=get_job_info(train)

In [19]:
len(train_job_info)

100000

In [20]:
def clean(x,df):
    res=[]
    n=len(x)
    d={}
    for i,content in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        data=df.iloc[i]
        job_id=data["job_id"]
        try:
            y=d[str(job_id)]
        except:
            content=content.replace(",","").replace("-"," ").replace("  "," ").replace("&eacute;","e")
            content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
            content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
            content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
            content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
            content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
            content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
            content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
            content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
            content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
            content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
            content=re.sub('<.*?>',"",content , flags=re.DOTALL)
            content=content.replace("&ldquo;","").replace("ldquo;","").replace("&rdquo;","").replace("rdquo;","")
            try:
                content=''.join([i for i in content if not i.isdigit()])
            except:
                pass
            content=content.split(" ")
            y=[]
            for i in content:
                if len(i)>1:
                    y.append(i.lower())
            d[str(job_id)]=y
        res.append(y)
    return res

In [21]:
cleaned_train_job_info=clean(train_job_info,train)

progress :  0.0
progress :  0.05
progress :  0.1
progress :  0.15
progress :  0.2
progress :  0.25
progress :  0.3
progress :  0.35
progress :  0.4
progress :  0.45
progress :  0.5
progress :  0.55
progress :  0.6
progress :  0.65
progress :  0.7
progress :  0.75
progress :  0.8
progress :  0.85
progress :  0.9
progress :  0.95


In [22]:
def remove_stopwords_job(x, df):
    res=[]
    n=len(x)
    d={}
    with open('stopwords.txt') as stopfile:
        stopwords = stopfile.read()
        stop = stopwords.split()
    for i,job in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        data=df.iloc[i]
        job_id=data["job_id"]
        try:
            a=d[str(job_id)]
        except:
            a=[]
            for word in job:
                if word not in stop:
                    a.append(word)
            d[str(job_id)]=a
        res.append(a)
    return res

In [23]:
def remove_stopwords_user(x, df):
    res=[]
    n=len(x)
    d={}
    with open('stopwords.txt') as stopfile:
        stopwords = stopfile.read()
        stop = stopwords.split()
    for i,user in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        data=df.iloc[i]
        user_id=data["user_id"]
        try:
            a=d[str(user_id)]
        except:
            a=[]
            for word in user:
                if word not in stop:
                    a.append(word)
            d[str(user_id)]=a
        res.append(a)
    return res

In [24]:
train_clean=remove_stopwords_job(cleaned_train_job_info,train)

progress :  0.0
progress :  0.05
progress :  0.1
progress :  0.15
progress :  0.2
progress :  0.25
progress :  0.3
progress :  0.35
progress :  0.4
progress :  0.45
progress :  0.5
progress :  0.55
progress :  0.6
progress :  0.65
progress :  0.7
progress :  0.75
progress :  0.8
progress :  0.85
progress :  0.9
progress :  0.95


In [25]:
train_user_clean=remove_stopwords_user(clean_train_user_info,train)

progress :  0.0
progress :  0.05
progress :  0.1
progress :  0.15
progress :  0.2
progress :  0.25
progress :  0.3
progress :  0.35
progress :  0.4
progress :  0.45
progress :  0.5
progress :  0.55
progress :  0.6
progress :  0.65
progress :  0.7
progress :  0.75
progress :  0.8
progress :  0.85
progress :  0.9
progress :  0.95


In [26]:
def get_occurence_job(x,df):
    res=[]
    n=len(x)
    d={}
    for i,job in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        data=df.iloc[i]
        job_id=data["job_id"]
        try:
            a=d[str(job_id)]
        except:
            a=dict(Counter(job))
            d[str(job_id)]=a
        res.append(a)
    return res

In [27]:
def get_occurence_user(x,df):
    res=[]
    n=len(x)
    d={}
    for i,user in enumerate(x):
        if i%5000==0:
            print("progress : ",i/n)
        data=df.iloc[i]
        user_id=data["user_id"]
        try:
            a=d[str(user_id)]
        except:
            a=dict(Counter(user))
            d[str(user_id)]=a
        res.append(a)
    return res

In [28]:
occurences=get_occurence_job(train_clean,train)

progress :  0.0
progress :  0.05
progress :  0.1
progress :  0.15
progress :  0.2
progress :  0.25
progress :  0.3
progress :  0.35
progress :  0.4
progress :  0.45
progress :  0.5
progress :  0.55
progress :  0.6
progress :  0.65
progress :  0.7
progress :  0.75
progress :  0.8
progress :  0.85
progress :  0.9
progress :  0.95


In [29]:
train_user_occurences=get_occurence_user(train_user_clean,train)

progress :  0.0
progress :  0.05
progress :  0.1
progress :  0.15
progress :  0.2
progress :  0.25
progress :  0.3
progress :  0.35
progress :  0.4
progress :  0.45
progress :  0.5
progress :  0.55
progress :  0.6
progress :  0.65
progress :  0.7
progress :  0.75
progress :  0.8
progress :  0.85
progress :  0.9
progress :  0.95


In [30]:
def get_vector_job(occurences, jobs, vector_size,df):
    vector=[]
    d={}
    model = Word2Vec.load('model.bin')
    nb=len(jobs)
    for i,job in enumerate(jobs):
        if i%10000==0:
            print("progress : ",i/nb )
        data=df.iloc[i]
        job_id=data["job_id"]
        try:
            res=d[str(job_id)]
        except:
            res=np.zeros(vector_size,)
            n=sum(list(occurences[i].values()))
            job_unique=list(set(job))
            for word in job_unique:
                try:
                    a=model[word]
                except:
                    #print(word)
                    a=None
                if a is not None:
                    count=occurences[i][word]
                    w=count/n
                    res+=w*a
            d[str(job_id)]=res
        vector.append(res)
    return vector

In [31]:
def get_vector_user(occurences, users, vector_size,df):
    vector=[]
    d={}
    model = Word2Vec.load('model.bin')
    nb=len(users)
    for i,user in enumerate(users):
        if i%10000==0:
            print("progress : ",i/nb )
        data=df.iloc[i]
        user_id=data["user_id"]
        try:
            res=d[str(user_id)]
        except:
            res=np.zeros(vector_size,)
            n=sum(list(occurences[i].values()))
            user_unique=list(set(user))
            for word in user_unique:
                try:
                    a=model[word]
                except:
                    #print(word)
                    a=None
                if a is not None:
                    count=occurences[i][word]
                    w=count/n
                    res+=w*a
            d[str(user_id)]=res
        vector.append(res)
    return vector

In [32]:
X=get_vector_job(occurences, train_clean, 200,train)

progress :  0.0


C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


progress :  0.1
progress :  0.2
progress :  0.3
progress :  0.4
progress :  0.5
progress :  0.6
progress :  0.7
progress :  0.8
progress :  0.9


In [33]:
X_user=get_vector_user(train_user_occurences, train_user_clean, 200, train)

progress :  0.0


C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


progress :  0.1
progress :  0.2
progress :  0.3
progress :  0.4
progress :  0.5
progress :  0.6
progress :  0.7
progress :  0.8
progress :  0.9


In [34]:
X_train=np.array(X)

In [35]:
X_train.shape

(100000, 200)

In [36]:
X_train_user=np.array(X_user)

In [37]:
X_train_user.shape

(100000, 200)

In [38]:
#X_train_job_final=np.concatenate((X_train, job_characteristics.values), axis=1)

In [39]:
#X_train_job_final.shape

In [40]:
#X_train_user_final=np.concatenate((X_train_user, user_characteristics.values), axis=1)

In [41]:
#X_train_user_final.shape

In [42]:
#pca = PCA(n_components=2)
#result_user = pca.fit_transform(X_train_user_final)

In [43]:
#print(pca.explained_variance_ratio_)

In [44]:
#pca = PCA(n_components=2)
#result = pca.fit_transform(X_train_job_final)

In [45]:
#print(pca.explained_variance_ratio_)

In [46]:
#users=list(train["user_id"].values)
#users=[str(i) for i in users]

In [47]:
def get_job_info_plot(df):
    job_info=[]
    for i in range(df.shape[0]):
        data=df.iloc[i]
        title=data["job_title"]
        em_title=data["employer_title"]
        a=""
        a+=title+" "
        a+=em_title+" "
        job_info.append(a)
    return job_info

In [48]:
#infos=get_job_info_plot(train)

In [49]:
"""k=200
plt.figure(figsize=(20,20))
plt.scatter(result_user[:k, 0], result_user[:k, 1])
for i, word in enumerate(users[:k]):
    plt.annotate(word, xy=(result_user[i, 0], result_user[i, 1]))
plt.show()"""

'k=200\nplt.figure(figsize=(20,20))\nplt.scatter(result_user[:k, 0], result_user[:k, 1])\nfor i, word in enumerate(users[:k]):\n    plt.annotate(word, xy=(result_user[i, 0], result_user[i, 1]))\nplt.show()'

In [50]:
train.iloc[10][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']]

job_title                                                Graduate Programs
employer_title                                               Credit Suisse
employer_sector_title                                   Investment Banking
job_sector_title                                        Financial Services
gender                                                                   F
ethnicity                                                              NaN
university_name                            Queen Mary University of London
degree_subject_name                                              Economics
user_preference          Consumer, FMCG & Retail Energy & Infrastructur...
Name: 5372, dtype: object

In [51]:
def get_similar_member(df, X_train, doc_id, n):
    res=[]
    beta=[]
    cos=list(cosine_similarity(X_train[doc_id,:].reshape(1, -1) , X_train))[0]
    most_similar_doc_id=np.argsort(cos)[::-1]
    a=0
    while cos[most_similar_doc_id[a]]>=0.9999999:
        a+=1
    most_similar_doc_id=most_similar_doc_id[a:]
    res.append(df.iloc[most_similar_doc_id[0]]["user_id"])
    beta.append(cos[most_similar_doc_id[0]])
    for i in range(1,len(most_similar_doc_id)):
        if cos[most_similar_doc_id[i]] not in beta:
            user=df.iloc[most_similar_doc_id[i]]["user_id"]
            if user not in res:
                res.append(user)
                beta.append(cos[most_similar_doc_id[i]])
    print(" user id : ", df.iloc[doc_id]["user_id"])
    print("most similar doc : ", res[:n])
    print("most similar values : ", beta[:n])
    print("not similar doc : ", res[-n:][::-1])
    print("not similar values : ", beta[-n:][::-1])

In [52]:
get_similar_member(train, X_train_user, 10, 5)

 user id :  67420
most similar doc :  [74199, 154654, 170872, 156985, 113673]
most similar values :  [0.9776953517269544, 0.9748114322881276, 0.9740408504458226, 0.9732623623336111, 0.9732582954136554]
not similar doc :  [158698, 173617, 193292, 169771, 167098]
not similar values :  [-0.10120783467405582, 0.02071661008772877, 0.11520878805496057, 0.14089604317197513, 0.1465722137056062]


# user

In [53]:
train[train["user_id"]==67420][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(10)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
5372,Graduate Programs,Credit Suisse,Investment Banking,Financial Services,F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."
5377,Professional Development Programme,State Street,Financial Services,Financial Services,F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."
5373,Management Trainee,Muntons,"Consumer, FMCG & Retail","Consumer, FMCG & Retail",F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."
5380,Finance Graduate Program,Amazon,"Consumer, FMCG & Retail",Financial Services,F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."
5374,New Analyst Programme,Goldman Sachs,Financial Services,Technology: Consulting & Project Management,F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."
5375,Counterparty Risk Management Trainee,Societe Generale,Financial Services,Financial Services,F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."
5379,Graduate Programme,Insight Investment,Investment & Asset Management,Financial Services,F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."
5378,Graduate Analyst Positions,Evercore,Financial Services,Financial Services,F,NaN,Queen Mary University of London,Economics,"Consumer, FMCG & Retail Energy & Infrastructur..."


# similar users

In [54]:
train[train["user_id"]==74199][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail()

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
23232,Deloitte Graduate Fast Track Programme 2018,Deloitte,Consulting,"Accounting, Tax & Audit",Female,NaN,University of Birmingham,Economics,"Commercial Law Consumer, FMCG & Retail Educati..."
23231,Asset & Wealth Management - Wealth Management ...,J.P. Morgan,Financial Services,Investment & Asset Management,Female,NaN,University of Birmingham,Economics,"Commercial Law Consumer, FMCG & Retail Educati..."
23229,Investment Graduate Scheme,M&G Investments,Financial Services,Financial Services,Female,NaN,University of Birmingham,Economics,"Commercial Law Consumer, FMCG & Retail Educati..."
23227,Graduate Programs,Credit Suisse,Investment Banking,Financial Services,Female,NaN,University of Birmingham,Economics,"Commercial Law Consumer, FMCG & Retail Educati..."
23228,New Analyst Programme,Goldman Sachs,Financial Services,Technology: Consulting & Project Management,Female,NaN,University of Birmingham,Economics,"Commercial Law Consumer, FMCG & Retail Educati..."


In [55]:
train[train["user_id"]==154654][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
85876,Finance Graduate Program,Amazon,"Consumer, FMCG & Retail",Financial Services,Female,White/White British,University of Cambridge,History,Investment Banking Insurance & Risk Management...
85864,Graduate Programs,Credit Suisse,Investment Banking,Financial Services,Female,White/White British,University of Cambridge,History,Investment Banking Insurance & Risk Management...
85869,Investment Banking Analyst Program - Full-time,J.P. Morgan,Financial Services,Investment Banking,Female,White/White British,University of Cambridge,History,Investment Banking Insurance & Risk Management...
85867,New Analyst Programme,Goldman Sachs,Financial Services,Technology: Consulting & Project Management,Female,White/White British,University of Cambridge,History,Investment Banking Insurance & Risk Management...
85875,Internships,Jane Street,Financial Services,Technology: Consulting & Project Management,Female,White/White British,University of Cambridge,History,Investment Banking Insurance & Risk Management...
85873,Graduate Analysts Programme,Nomura,Financial Services,Technology: Consulting & Project Management,Female,White/White British,University of Cambridge,History,Investment Banking Insurance & Risk Management...
85860,Graduate Programme,DC Advisory Partners Limited,Financial Services,Financial Services,Female,White/White British,University of Cambridge,History,Investment Banking Insurance & Risk Management...


# not similar users

In [56]:
train[train["user_id"]==158698][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
106992,Herbert Smith Freehills Event Dates 2017-2018,Herbert Smith Freehills,Commercial Law,Commercial Law,Male,Asian/Asian British,University of Warwick,Engineering - Mechanical,Commercial Law


In [57]:
train[train["user_id"]==173617][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
26048,Herbert Smith Freehills Event Dates 2017-2018,Herbert Smith Freehills,Commercial Law,Commercial Law,Female,White/White British,University College London (UCL),Translation,NaN
26047,Vacation Scheme,Gowling WLG,Commercial Law,Commercial Law,Female,White/White British,University College London (UCL),Translation,NaN


In [70]:
train[train["user_id"]==98253][['job_title','employer_title', 'employer_sector_title', 'job_sector_title', 'gender',
       'ethnicity', 'university_name', 'degree_subject_name', 'user_preference']].tail(7)

,job_title,employer_title,employer_sector_title,job_sector_title,gender,ethnicity,university_name,degree_subject_name,user_preference
125035,Campus Ambassadors,Reed Smith,Commercial Law,Commercial Law,Female,NaN,Durham University,History and Spanish,Commercial Law


In [58]:
def get_similar_doc(X_train, doc_id, n):
    res=[]
    beta=[]
    cos=list(cosine_similarity(X_train[doc_id,:].reshape(1, -1) , X_train))[0]
    most_similar_doc_id=np.argsort(cos)[::-1]
    a=0
    while cos[most_similar_doc_id[a]]>=0.9999999:
        a+=1
    most_similar_doc_id=most_similar_doc_id[a:]
    res.append(most_similar_doc_id[0])
    beta.append(cos[most_similar_doc_id[0]])
    for i in range(1,len(most_similar_doc_id)):
        if cos[most_similar_doc_id[i]] not in beta:
            res.append(most_similar_doc_id[i])
            beta.append(cos[most_similar_doc_id[i]])
    print("most similar doc : ", res[:n])
    print("most similar values : ", beta[:n])
    print("not similar doc : ", res[-n:][::-1])
    print("not similar values : ", beta[-n:][::-1])

In [59]:
def clean_similarity(x):
    res=[] 
    for content in x:
        content=content.replace(",","").replace("-"," ").replace("  "," ")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=content.replace("&bull;","\r\n ").replace("bull;","\r\n ").replace("\r\r\n\t"," ").replace("\r\r\n"," ")
        content=content.replace("\r\r\n\r\r\n"," ").replace("..."," ").replace("&#39;s","").replace(":","")
        content=content.replace("&eacute;","e").replace("&lsquo;","").replace("lsquo;","").replace("&#39;","").replace("#39;s","")
        content=content.replace("&nbsp;"," ").replace("nbsp;"," ").replace("&#39;","").replace("amp;","").replace("&middot;","")
        content=content.replace("."," ").replace(",","").replace("?"," ").replace("!","").replace("  "," ").replace("middot;","")
        content=content.replace("&rsquo;","").replace("rsquo;"," ").replace("\t"," ").replace("\r\n\r\n","").replace(" \r\n ","")
        content=content.replace("&ndash;"," ").replace("ndash;"," ").replace("\r\n\r\n"," ").replace(" \r\n "," ").replace("\r\n"," ")
        content=content.replace("\r"," ").replace("  ","").replace("   ","").replace("  ","").replace("   ","")
        content=content.replace("(","").replace(")","").replace(" - "," ").replace(" / "," ").replace("/"," ").replace("   ","")
        content=re.sub('<.*?>',"",content , flags=re.DOTALL)
        res.append(content)
    return res

In [61]:
data_similarity=clean_similarity(train_job_info)

In [62]:
get_similar_doc(X_train, 10, 5)

most similar doc :  [69307, 77022, 42907, 75232, 26673]
most similar values :  [0.939602703820889, 0.9389237348812738, 0.9260504443605474, 0.9151344383612695, 0.8944081509168142]
not similar doc :  [8555, 74064, 60679, 69981, 9685]
not similar values :  [-0.23217292623603597, -0.1338385141763883, -0.03124304928293687, 0.07172323018953197, 0.1197311342829278]


# job

In [63]:
data_similarity[10]

'Graduate Programs Financial Services Training We provide a rich development environment by combining formal learning on the job practice and mentoring  We believe soft skills are crucial to our business Thats why our training programs also develop your presentation communication and interpersonal skills You can expect exceptional support with any mandatory regulatory training and exams  Throughout your training youll have exposure to senior managers colleagues and peers across the bank at all levels which will add new insights and fresh perspectives to your understanding of what we do and what you need to do well at Credit Suisse   Receive technical and financial training from some of the industrys top instructors  Listen to business presentations from Credit Suisse professionals  Participate in philanthropic and teambuilding events  Attend social and networking events with peers and Credit Suisse professionals  Prepare for FCA regulatory training and exams  Rotate through the Shared 

# Similar jobs

In [64]:
data_similarity[69307]

'Summer Internship Programs Financial Services Summer Internship Program Our Summer Internship Program is designed to offer real insight into our business The program typically lasts ten weeks and is one of the most in depth internships within the financial services industry From day one youll be part of the team Youll face real challenges have client exposure enjoy real achievements and have your talents recognized every step of the way  Youll be responsible for projects and tasks that matter to the business Youll grow in knowledge skills and confidence Above all youll experience the team spirit that makes Credit Suisse such a special place to work Our program managers will help guide you through your summer experience and each intern is assigned a mentor and a buddy so youll have all the support you need  If you show real potential and demonstrate exceptional performance during your time with us you may be offered a full time position for the following year   Train in a comprehensive

# not similar jobs

In [66]:
data_similarity[74064]

'TPP Top 50 Oxford events Technology Consulting & Project Management On Tuesday 31st October we are hosting evening reception with free drinks and food at the Bodleian Library featuring renowned stand up comedian Sean Lock As well as live entertainment the reception will give you the opportunity to chat to TPP Oxford alumni and find out what it is like to work at the heart of a fast paced technology company  Job interviews will take place at the Examination Schools in Oxford on Thursday 2nd and Friday 3rd November giving you the opportunity to walk away with a &pound;40000 graduate job by the end of the week Oxford Events October  17th October Drinks reception at Lady Margaret Hall When 7 30pm 10pm  Where Talbot Hall 18th October Physics Society science themed quiz night free pizza & drinks When 5 15pm Where Physics Society 18th October Oxford Union event with JJ Abrams When 8pm Where Oxford Union 23rd October Evening reception at St Hughs College When 7 30pm 10pm Where Wordsworth Room

In [ ]:
data = np.concatenate((X_train, X_train_user), axis=1)

In [86]:
#data=data.reshape(data.shape[0],data.shape[1],1)

In [87]:
data.shape

(100000, 400, 1)

In [88]:
data[0].shape

(400, 1)

# Autoencoder

In [72]:
from keras.layers import Input, Dense
from keras.models import Model

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [78]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data, data, test_size=0.3)

In [93]:
input_img = Input(shape=(data.shape[1],))
encoded = Dense(200, activation='relu')(input_img)
encoded = Dense(100, activation='relu')(encoded)
encoded = Dense(50, activation='relu')(encoded)

decoded = Dense(100, activation='relu')(encoded)
decoded = Dense(200, activation='relu')(decoded)
decoded = Dense(400, activation='sigmoid')(decoded)

In [94]:
autoencoder = Model(data, decoded)
autoencoder.compile(loss='mean_squared_error', optimizer='sgd')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=500,
                shuffle=True,
                validation_data=(x_test, x_test))

TypeError: unhashable type: 'numpy.ndarray'